In [25]:
import numpy as np
import pandas as pd
import pymongo
import bson
from tqdm import tqdm

In [26]:
class MongoServer():
    credentials = None
    conn = None
    db = None
    collections = {}
    def __init__(self, run = False, credentials = "credentials.txt", db_name = "agile_data_science_group_3"):
        if run:
            if not self.connect2Mongo(credentials):
                print("Connection to server Failed.")
            
            if not self.connect2DataBase(db_name):
                print("Connection Data Base Failed.")
    """Rotine to connect to Mongo DB"""    
    def connect2Mongo(self, credentials = "credentials.txt"):
        try:
            #use your database name, user and password here:
            with open(credentials, 'r', encoding='utf-8') as f:
                [name,password,url,dbname]=f.read().splitlines()
            conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
            self.conn = conn
            return True
        except pymongo.errors.ConnectionFailure as e:
            return False
    
    """Routine to connect to a Data Base"""
    def connect2DataBase(self, db_name = "agile_data_science_group_3"):
        try:
            self.db = self.conn[db_name]
            return True
        except:
            return False
    
    """Return the available collections in a data base"""
    def listOfCollections (self):
        return self.db.collection_names()
    
    """Donwload all the ollections from the data base"""
    def getAllCollections (self):
        collections = self.listOfCollections()
        for col in collections:
            self.getCollectionFromServer(col)
        return True
    
    
    """Routine to get one collection of the Data Base"""
    # return the collection
    def getCollectionFromServer (self, name_collection):
        if name_collection in self.db.collection_names():
            self.collections[name_collection] = self.db.get_collection(name_collection)
            print ("Collection ",name_collection," Update in Local.")
            return True
        return False
    
    """ Take the collection from the local copy"""
    def getCollection(self, name_collection):
        if not name_collection in self.collections:
            if not self.getCollectionFromServer(name_collection):
                return False
        return self.collections[name_collection]
    
    """ Gets all the items of the collection"""
    def getItems(self, name_collection):
        if not name_collection in self.collections:
            if not self.getCollectionFromServer(name_collection):
                return False
        return [element for element in self.collections[name_collection].find()]
    
    """Query in one Collection"""
    def searchInCollection(self, name_collection, field, patro):
        return [element for element in self.collections[name_collection].find({field:patro})]
    
    """Query in all the Collections"""
    def searchInDB (self, field, patro):
        query = {}
        for collection in self.db.collection_names():
            query[collection] = [element for element in self.db.get_collection(collection).find({field:patro})]
        return query
    
    """Find one in the collection"""
    def findOne(self, collection_name):
        return self.db.get_collection(collection_name).find_one()
    
    """Froce Brute search"""
    def searchInDBForceBrute(self, patro):
        query = {}
        for collection in self.db.collection_names():
            query[collection] = []
            
            self.searchInCollection(collection, )
            print(self.getItems(collection))
            for element in self.getItems(collection)[0]:
                for key in element.keys():
                    query[collection] = [element for element in self.db.get_collection(collection).find({field:patro})]
                    
    def findNElement(self, collection_name, N):
        query = []
        for item in self.db.get_collection(collection_name).find().limit( N ):
            query.append(item)
        return query
    
    def insertInCollection (self, collection_name,  item):
        assert type(item) == dict, "Item must be a dictionary"
        return self.db.get_collection(collection_name).insert_one(item)
        
                
        
MS = MongoServer(True)   


In [52]:
MS.searchInCollection('recipes', 'id', name)[0]['_id']

ObjectId('59fb0d77eee3642f3c0684bd')

In [ ]:
for item in tqdm(MS.getItems('recipes_ingredients')):
    ingredients = list(item.values())[1]
    name = list(item)[1]
    recepie_id = MS.searchInCollection('recipes', 'id', name)[0]['_id']
    
    MS.insertInCollection('RecIng', {'recipe_id': recepie_id, 'ingredients':ingredients})

 37%|█████████████████████████████                                                  | 909/2467 [01:53<03:15,  7.98it/s]

In [49]:
MS.findOne('recipes')

{'_id': ObjectId('59fb0d77eee3642f3c0683f7'),
 'energy_KCal': 431,
 'fat_g': '16.6',
 'id': '20-minute-carbonara-with-peas',
 'image': 'https://be35832fa5168a30acd6-5c7e0f2623ae37b4a933167fe83d71b5.ssl.cf3.rackcdn.com/1602/20-minute-carbonara-with-peas__square.jpg',
 'list_ingredients': ['200g spaghetti',
  '250g frozen peas',
  '2 medium eggs',
  '25g Parmesan(grated)',
  '4 level tbsp reduced-fat crème fraîche',
  '1 tbsp olive oil',
  '4 rashers Asda Extra Special Unsmoked Back Bacon(chopped)',
  '1 shallot(finely chopped)',
  '1 garlic clove(crushed)',
  'Chopped parsley(to serve)'],
 'name': '20-minute carbonara with peas',
 'ningredients': 10.0,
 'price_p': '115',
 'salt_mg': '1805',
 'saturated_fat_g': '6.1',
 'serves': '4',
 'sugar_g': '5',
 'time_min': '20',
 'url': 'https://www.asdagoodliving.co.uk/food/recipes/20-minute-carbonara-with-peas'}